In [123]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import tensorflow as tf
from tensorflow.keras import layers , activations , models , preprocessing, utils
import re

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../../dados/portuguese/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

../../dados/portuguese/compliment.yml
../../dados/portuguese/conversations.yml
../../dados/portuguese/games.yml
../../dados/portuguese/greetings.yml
../../dados/portuguese/linguistic_knowledge.yml
../../dados/portuguese/money.yml
../../dados/portuguese/proverbs.yml
../../dados/portuguese/suggestions.yml
../../dados/portuguese/trivia.yml
../../dados/portuguese/unilab.yml


In [2]:
import yaml

dir_path = '../../dados/portuguese/'
files_list = os.listdir(dir_path + os.sep)

In [68]:
questions = []
answers = []

for filepath in files_list:
    file_ = open(dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(file_)
    conversations = docs['conversations']
    print("*"*80)
    print(conversations)
    for con in conversations:
        if len(con) > 2 :
            questions.append(con[0])
            replies = con[1 :]
            ans = ''
            for rep in replies:
                ans += ' ' + rep
            answers.append(ans)
        elif len(con)> 1:
            questions.append(con[0])
            answers.append(con[1])

********************************************************************************
[['você é linda', 'eu sei'], ['adoro você', 'também'], ['gosto da sua sinceridade', 'que bom'], ['você é valente', 'obrigado'], ['você é muito sexy', 'você é que é'], ['gosto do seu talento', 'que bom'], ['você é de mais', 'sim, eu sei'], ['você é uma graça', 'nem tanto'], ['como você é animada', 'é bom para saúde ser animada'], ['tu és insubstituível', 'ninguém é insubstituível'], ['tu trouxeste luz a minha vida', 'você é que trouxe luz a minha vida'], ['eu amo os teus expressivos olhos doces', 'eu te amo por completo'], ['estou muito orgulhoso de ti', 'que bom, obrigada'], [' estou muito orgulhoso de você', 'obrigada'], ['estou muito orgulhosa de ti', 'que bom, obrigado'], ['estou muito orgulhosa de você', 'obrigado'], ['você é tão engraçado', 'você acha?'], ['você é ótimo na cama', 'que bom'], ['você é ótima na cama', 'que bom'], ['você está linda', 'obrigada'], ['você está lindo', 'você é que está'], [

In [69]:
answers_with_tags = []
for i in range(len(answers)):
    if type(answers[i]) == str:
        answers_with_tags.append(answers[i])
    else:
        questions.pop(i)

answers = []
for i in range(len(answers_with_tags)) :
    answers.append('<START> ' + answers_with_tags[i] + ' <END>')

tokenizer = preprocessing.text.Tokenizer()
# tokenizer = layers.TextVectorization()
tokenizer.fit_on_texts(questions + answers)
VOCAB_SIZE = len(tokenizer.word_index)+1

In [71]:
answers

['<START> eu sei <END>',
 '<START> também <END>',
 '<START> que bom <END>',
 '<START> obrigado <END>',
 '<START> você é que é <END>',
 '<START> que bom <END>',
 '<START> sim, eu sei <END>',
 '<START> nem tanto <END>',
 '<START> é bom para saúde ser animada <END>',
 '<START> ninguém é insubstituível <END>',
 '<START> você é que trouxe luz a minha vida <END>',
 '<START> eu te amo por completo <END>',
 '<START> que bom, obrigada <END>',
 '<START> obrigada <END>',
 '<START> que bom, obrigado <END>',
 '<START> obrigado <END>',
 '<START> você acha? <END>',
 '<START> que bom <END>',
 '<START> que bom <END>',
 '<START> obrigada <END>',
 '<START> você é que está <END>',
 '<START> obrigado <END>',
 '<START> obrigada <END>',
 '<START> estudo para isso <END>',
 '<START> é sim <END>',
 '<START> você também <END>',
 '<START> você também é maravilhosa <END>',
 '<START> obrigada <END>',
 '<START> você também fala bem <END>',
 '<START> tenho sim <END>',
 '<START> obrigado <END>',
 '<START> obrigada <EN

In [5]:
VOCAB_SIZE

1381

In [95]:
from gensim.models import Word2Vec
import re

vocab = list(tokenizer.word_index.keys())

def tokenize(sentences):
    tokens_list = []
    vocabulary = []
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub('[^a-zA-Z]', ' ', sentence)
        tokens = sentence.split()
        vocabulary += tokens
        tokens_list.append(tokens)
    return tokens_list , vocabulary

In [96]:
tokenized_questions = tokenizer.texts_to_sequences(questions)
maxlen_questions = max([len(x) for x in tokenized_questions])
# padded_questions = preprocessing.sequence.pad_sequences(tokenized_questions , maxlen=maxlen_questions , padding='post')
# encoder_input_data = np.array(padded_questions)
encoder_input_data = preprocessing.sequence.pad_sequences(tokenized_questions , maxlen=maxlen_questions , padding='post')

In [97]:
print(encoder_input_data.shape)

(280, 28)


In [98]:
tokenized_answers = tokenizer.texts_to_sequences(answers)
maxlen_answers = max([len(x) for x in tokenized_answers])
# padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers , maxlen=maxlen_answers , padding='post')
# decoder_input_data = np.array(padded_answers)
decoder_input_data = preprocessing.sequence.pad_sequences(tokenized_answers , maxlen=maxlen_answers , padding='post')

In [89]:
print(decoder_input_data.shape)

(280, 182)


In [101]:
VOCAB_SIZE

1382

In [124]:
tokenized_answers = tokenizer.texts_to_sequences(answers)
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers , maxlen=maxlen_answers , padding='post')
decoder_output_data = utils.to_categorical(padded_answers , VOCAB_SIZE)
# onehot_answers = utils.to_categorical(padded_answers , VOCAB_SIZE)
# decoder_output_data = np.array(onehot_answers)

In [126]:
tokenized_answers[10]

[7, 3, 5, 315, 162, 8, 52, 70, 2]

In [106]:
print(decoder_output_data.shape)

(280, 182, 1382)


In [107]:
# Embedding, LSTM and Dense layers
encoder_inputs = tf.keras.layers.Input(shape=(maxlen_questions ,))
encoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, 200 , mask_zero=True) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM(200 , return_state=True)(encoder_embedding)
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=(maxlen_answers , ))
decoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM(200 , return_state=True , return_sequences=True)
decoder_outputs , _ , _ = decoder_lstm (decoder_embedding , initial_state=encoder_states)
decoder_dense = tf.keras.layers.Dense(VOCAB_SIZE , activation=tf.keras.activations.softmax) 
output = decoder_dense (decoder_outputs)

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output)
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

In [108]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 28)]                 0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, 182)]                0         []                            
                                                                                                  
 embedding_2 (Embedding)     (None, 28, 200)              276400    ['input_5[0][0]']             
                                                                                                  
 embedding_3 (Embedding)     (None, 182, 200)             276400    ['input_6[0][0]']             
                                                                                            

In [117]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=32, epochs=200) 

Epoch 1/200
9/9 [==============================] - 5s 544ms/step - loss: 4.4642
Epoch 2/200
9/9 [==============================] - 5s 549ms/step - loss: 4.4260
Epoch 3/200
9/9 [==============================] - 5s 510ms/step - loss: 4.3740
Epoch 4/200
9/9 [==============================] - 5s 510ms/step - loss: 4.3738
Epoch 5/200
9/9 [==============================] - 5s 508ms/step - loss: 4.3226
Epoch 6/200
9/9 [==============================] - 5s 508ms/step - loss: 4.3309
Epoch 7/200
9/9 [==============================] - 5s 513ms/step - loss: 4.3215
Epoch 8/200
9/9 [==============================] - 5s 506ms/step - loss: 4.3106
Epoch 9/200
9/9 [==============================] - 5s 519ms/step - loss: 4.2499
Epoch 10/200
9/9 [==============================] - 5s 521ms/step - loss: 4.2381
Epoch 11/200
9/9 [==============================] - 5s 507ms/step - loss: 4.2312
Epoch 12/200
9/9 [==============================] - 5s 504ms/step - loss: 4.1949
Epoch 13/200
9/9 [===================

9/9 [==============================] - 5s 535ms/step - loss: 2.4220
Epoch 103/200
9/9 [==============================] - 5s 525ms/step - loss: 2.4167
Epoch 104/200
9/9 [==============================] - 5s 513ms/step - loss: 2.3958
Epoch 105/200
9/9 [==============================] - 5s 523ms/step - loss: 2.3973
Epoch 106/200
9/9 [==============================] - 5s 507ms/step - loss: 2.3757
Epoch 107/200
9/9 [==============================] - 5s 517ms/step - loss: 2.3319
Epoch 108/200
9/9 [==============================] - 5s 510ms/step - loss: 2.3384
Epoch 109/200
9/9 [==============================] - 5s 508ms/step - loss: 2.3125
Epoch 110/200
9/9 [==============================] - 5s 506ms/step - loss: 2.2985
Epoch 111/200
9/9 [==============================] - 5s 504ms/step - loss: 2.2919
Epoch 112/200
9/9 [==============================] - 5s 521ms/step - loss: 2.2651
Epoch 113/200
9/9 [==============================] - 5s 508ms/step - loss: 2.2206
Epoch 114/200
9/9 [===========

In [118]:
def inference():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=(200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=(200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [119]:
def preprocess_input(input_sentence):
    tokens = input_sentence.lower().split()
    tokens_list = []
    for word in tokens:
        tokens_list.append(tokenizer.word_index[word]) 
    return preprocessing.sequence.pad_sequences([tokens_list] , maxlen=maxlen_questions , padding='post')

In [120]:
enc_model , dec_model = inference()

In [121]:
tokenizer.word_index['robô']

820

In [122]:
tests = ['oi', 'Você é um robô', 'o que é linguística', 'o que é um opinião', 'até mais']

for i, frase in enumerate(tests):
    states_values = enc_model.predict(preprocess_input(tests[i]))
    empty_target_seq = np.zeros((1 , 1))
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([empty_target_seq] + states_values)
        sampled_word_index = np.argmax(dec_outputs[0, -1, :])
        sampled_word = None
        
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += f' {word}'
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros((1 , 1))  
        empty_target_seq[0 , 0] = sampled_word_index
        states_values = [h , c] 
    print(f'Human: {tests[i]}')
    print()
    decoded_translation = decoded_translation.split(' end')[0]
    print(f'Bot: {decoded_translation}')
    print('-'*25)

1/1 [==============================] - 0s 22ms/step
Human: oi

Bot:  olá
-------------------------
1/1 [==============================] - 0s 20ms/step
Human: Você é um robô

Bot:  você é que é
-------------------------
1/1 [==============================] - 0s 21ms/step
Human: o que é linguística

Bot:  bom a alfabetização é a ciência que estuda os fenômenos relacionados e o nome em um nome é a menos que não seja holandês agora
-------------------------
1/1 [==============================] - 0s 21ms/step
Human: o que é um opinião

Bot:  bem a alfabetização pode dar fazer à margem da instituição escolar e o nome em um nome em um livro cujo nome
-------------------------
1/1 [==============================] - 0s 21ms/step
Human: até mais

Bot:  até
-------------------------
